In [1]:
# Import required libraries
import numpy as np
import math
import datetime as dt
import pandas as pd
import pathlib

In [48]:
# Load data
# get relative data folder
PATH = pathlib.Path('./data').parent
DATA_PATH = PATH.joinpath("data").resolve()
PIDAR = pd.read_csv(DATA_PATH.joinpath("pidar.csv"))
AT = pd.read_csv(DATA_PATH.joinpath("DS_Asistencia_Tecnica.csv"))
EVA = pd.read_csv(DATA_PATH.joinpath('EVA_cultivos_modificado.csv'))
url = 'https://raw.githubusercontent.com/ds4a-team-envigado/project/master/Dashboard/VERSION%202/data/AP_SABANAGENERAL.csv'
SG = pd.read_csv(url, index_col=0,sep=";",encoding='latin-1')
CP = pd.read_csv(DATA_PATH.joinpath('cadenas_productivas_ADR.csv'), encoding='UTF-8')
dfs = [PIDAR,AT,EVA,SG]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 209: invalid start byte

In [7]:
PIDAR.head()

,Unnamed: 0,Unnamed: 0.1,BP,No. VP,Nombre Proyecto,Departamento,Municipio,Total beneficiarios,Hombres,Mujeres,...,Valor total cofinanciación ADR,Valor Contrapartida,Valor total proyecto,Hectáreas,Resolución,Fecha resolución,Vigencia,Lineamiento de consejo directivo,AP_ID_ADR,AP_CADENA_PRODUCTIVA_ADR
0,5,5,6,6,Mejorar las capacidades técnicas de adecuación...,CHOCO,CARMEN DEL DARIÉN,35,30,5,...,250000000,513500000,763500000,100,1219,06/29/2017 12:00:00 AM,2017,NaN,1,AGRICOLA
1,16,16,17,17,Fortalecer la capacidad productiva y manejo de...,META,MESETAS,40,31,9,...,500000000,106600000,606600000,40,1250,07/12/2017 12:00:00 AM,2017,NaN,1,AGRICOLA
2,19,19,20,20,Fortalecer la productividad y competitividad d...,NORTE DE SANTANDER,TIBÚ,46,31,15,...,252785478,351466000,604251478,92,1453,10/12/2017 12:00:00 AM,2017,NaN,1,AGRICOLA
3,22,22,23,23,Fortalecer la cadena productiva de café y plát...,ANTIOQUIA,DABEIBA,25,20,5,...,401373250,355140000,756513250,69,1351,08/28/2017 12:00:00 AM,2017,NaN,1,AGRICOLA
4,24,24,25,25,Fortalecimiento de la productividad del cultiv...,META,LA MACARENA,28,16,12,...,255000000,41570000,296570000,93,1401,09/19/2017 12:00:00 AM,2017,NaN,1,AGRICOLA


In [3]:
# #Agrupar por departamtento cada una de las columnas
dff = EVA.groupby(['DEPARTAMENTO','YEAR','CADENA_PRODUCTIVA_ADR']).sum().reset_index()
dff.head()

,DEPARTAMENTO,YEAR,CADENA_PRODUCTIVA_ADR,COD_DEP,COD_MUN,AREA_SEMBR,AREA_COSECH,PRODUCCION,RENDIMIENTO,NIVEL,ID_ADR
0,AMAZONAS,2006,ARROZ,273,274339,11,11,16,4.35,9,37527
1,AMAZONAS,2006,MAIZ,455,457151,204,203,468,7.86,15,62520
2,AMAZONAS,2007,ACHICORIA,182,182541,6,6,6,2.00,6,21610
3,AMAZONAS,2007,AHUYAMA,91,91540,10,10,38,3.80,3,11213
4,AMAZONAS,2007,AJI,182,182541,28,28,35,2.50,6,22418


In [4]:
AT.head()

,Unnamed: 0,Id_Usuario,Sexo,Tipo_Poblacional,Discapacidad,Departamento,Municipio,Vereda,Predio,Cadena_Productiva,...,NA4,C1A5,C2A5,C3A5,C4A5,NA5,CLAS_GRAL,CADENA_PRODUCTIVA_ADR,ID_ADR,Tipo de proyecto
0,0,1,M,NINGUNA,NO,BOLIVAR,13838,CARABALI,PIE DE MONTE,MAIZ,...,3,3.0,3.0,3.0,3.0,3,2,MAIZ,12504.0,Asistencia Tecnica
1,3,4,M,NINGUNA,NO,BOLIVAR,13647,PATON,PATON,MAIZ,...,1,1.0,1.0,1.0,1.0,1,1,MAIZ,12504.0,Asistencia Tecnica
2,8,9,M,NINGUNA,NO,BOLIVAR,13248,La Enea,Sin nombre,MAIZ,...,1,1.0,1.0,1.0,1.0,1,1,MAIZ,12504.0,Asistencia Tecnica
3,10,11,M,NINGUNA,NO,BOLIVAR,13248,La Enea,Sin nombre,MAIZ,...,1,1.0,1.0,1.0,1.0,1,1,MAIZ,12504.0,Asistencia Tecnica
4,13,14,M,NINGUNA,NO,BOLIVAR,13780,PATICO,LAS PALMAS,MAIZ,...,1,1.0,1.0,1.0,1.0,1,1,MAIZ,12504.0,Asistencia Tecnica


In [22]:
import gzip
import json
import string

import nltk # imports the natural language toolkit
import pandas as pd
import plotly as px
import matplotlib as plt

import nltk # imports the natural language toolkit
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
import numpy  as np
import string
import plotly
from nltk.stem import PorterStemmer 
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()

nltk.download('punkt')

CP_ADR = EVA['CADENA_PRODUCTIVA_ADR'].unique()

[nltk_data] Downloading package punkt to C:\Users\Carlos Loaiza
[nltk_data]     Velez\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Carlos Loaiza
[nltk_data]     Velez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Carlos Loaiza
[nltk_data]     Velez\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [38]:
def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

In [29]:
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    s = s.replace(' ', '')
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s    

In [37]:
import re
from unicodedata import normalize

word_list = []

for sentence in PIDAR['Nombre Proyecto']:
    word_list = word_list + re.split(r',', sentence)
    
word_list2 = []
for word in word_list:
    word = word.lstrip().rstrip().upper()
    word = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", word), 0, re.I)
    word_list2.append(word)

word_list2 = list(dict.fromkeys(word_list2))
word_list2.sort()

In [52]:
# clean text data
import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

PIDAR['TITULO'] = PIDAR['Nombre Proyecto'].apply(lambda x: clean_text(x))

In [54]:
CP = []
for i in EVA['CADENA_PRODUCTIVA_ADR'].unique():
    dep = normalize(i)
    new_df.append(dep)
CP = pd.concat(new_df)

TypeError: normalize() takes exactly 2 arguments (1 given)

In [53]:
PIDAR['TITULO'] = normalize(PIDAR['TITULO'])

TypeError: normalize() takes exactly 2 arguments (1 given)